In [2]:
# merge surface shapefiles

import os
from os.path import isfile, join

base_dir = "C:/Users/markegge/work/wells"
output_file = "abandoned-wells.shp"



In [ ]:
# merge shapefiles
shape_files = [f for f in os.listdir(base_dir) if isfile(join(base_dir, f)) and f[-5:] == 's.shp']

if os.path.isfile(output_file):
    raise IOError('output file already exists')
        
for file_name in shape_files:
    if not os.path.isfile(output_file):
        command = 'ogr2ogr -f "ESRI Shapefile" -a_srs NAD27 %s %s' % (output_file, file_name)
    else:
        command = 'ogr2ogr -f "ESRI Shapefile" -update -append %s %s' % (output_file, file_name)
    !$command
print 'shapefile merged into %s' % output_file

In [ ]:
# Convert shapefile to geojson
command = "ogr2ogr -f GeoJSON -t_srs crs:84 %s %s" % (output_file[:-4] + '.geojson', output_file)
!$command

In [4]:
# Convert geojson to bin
import array, csv, datetime, json, math, random

input_file = 'wells.geojson'
output_file = 'abandoned-ft.bin'

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def YearMonthDayToEpoch(year, month, day):
  return (datetime.datetime(int(year), int(month), int(day)) - datetime.datetime(1970, 1, 1)).total_seconds()

def LonLatToECEF(lon,lat, elv = 0):
    lat = lat * (math.pi/180)
    lon = lon * (math.pi/180)
    radius = (6.371e6 + elv) / 6.371e6
    x = -radius * math.cos(lat) * math.sin(lon)
    y = radius * math.sin(lat)
    z = -radius * math.cos(lat)*math.cos(lon)
    return [x,y,z]

well_synnum = [4, 5, 6] # active wells - see list below
abandoned_status = [7,8,10,19,20]

# Convert GeoJSON file to bin file in with x,y in Web Mercator
f = open(input_file)
geojson = json.load(f)
data = []
for feature in geojson['features']:
    geometry = feature['geometry']
    properties = feature['properties']
    if properties['SYMNUM'] in abandoned_status:
        lon = properties['LONG']
        lat = properties['LAT']
        x,y = LonLatToPixelXY([lon,lat])
        year = 2015
        month = random.randrange(1,12,1) # bogus month
        day = random.randrange(1,28,1) # bogus day
        epochtime = YearMonthDayToEpoch(year, month, day)
        data += [x,y,epochtime]

f.close()
array.array('f', data).tofile(open(output_file, 'wb'))


In [ ]:
'''
Attribute_Label: SYMNUM
Attribute_Definition: Indicates the type of symbol to display for a point feature

Value	Definition
2	Permitted Location
3	Dry Hole
4	Oil Well
5	Gas Well
6	Oil/Gas Well
7	Plugged Oil Well
8	Plugged Gas Well
9	Canceled Location
10	Plugged Oil/Gas Well
11	Injection/Disposal Well
12	Core Test
13	Directional Surface Location
15	Radioactive Well
16	Sulfur Core Test
17	Storage from Oil
18	Storage from Gas
19	Shut-In Well (Oil)
20	Shut-In Well (Gas)
21	Injection/Disposal From Oil
22	Injection/Disposal From Gas
23	Injection/Disposal From Oil/Gas
24	Offshore Platform
25	Horizontal Drainhole Line
36	Geothermal Well
42	Directional Well Line
43	Directional Well Line
73	Brine Mining Well
74	Water Supply Well
75	Water Supply from Oil
76	Water Supply from Gas
77	Water Supply from Oil/Gas
78	Observation Well
79	Observation from Oil
80	Observation from Gas
81	Observation from Oil/Gas
86	Horizontal Drainhole
87	Sidetrack Well Surface Location
88	Storage Well
89	Service Well
90	Service from Oil
91	Service from Gas
92	Service from Oil/Gas
103	Storage from Oil/Gas
104	Injection/Disposal from Storage
105	Injection/Disposal from Storage/Oil
106	Injection/Disposal from Storage/Gas
107	Injection/Disposal from Storage/Oil/Gas
108	Observation from Storage
109	Observation from Storage/Oil
110	Observation from Storage/Gas
111	Observation from Storage/Oil/Gas
112	Service from Storage
113	Service from Storage/Oil
114	Service from Storage/Gas
115	Service from Storage/Oil/Gas
116	Plugged Storage
117	Plugged Storage/Oil
118	Plugged Storage/Gas
119	Plugged Storage/Oil/Gas
121	Brine Mining from Oil
122	Brine Mining from Gas
123	Brine Mining from Oil/Gas
124	Injection/Disposal from Brine Mining
125	Injection/Disposal from Brine Mining/Oil
126	Injection/Disposal from Brine Mining/Gas
127	Injection/Disposal from Brine Mining/Oil/Gas
128	Observation from Brine Mining
129	Observation from Brine Mining/Oil
130	Observation from Brine Mining/Gas
131	Observation from Brine Mining/Oil/Gas
132	Service from Brine Mining
133	Service from Brine Mining/Oil
134	Service from Brine Mining/Gas
135	Service from Brine Mining/Oil/Gas
136	Plugged Brine Mining
137	Plugged Brine Mining/Oil
138	Plugged Brine Mining/Gas
139	Plugged Brine Mining/Oil/Gas
140	Storage/Brine Mining
141	Storage/Brine Mining/Oil
142	Storage/Brine Mining/Gas
143	Storage/Brine Mining/Oil/Gas
144	Inj/Disposal from Storage/Brine Mining
145	Inj/Disposal from Storage/Brine Mining/Oil
146	Inj/Disposal from Storage/Brine Mining/Gas
147	Inj/Disposal from Storage/Brine Mining/Oil/Gas
148	Observation from Storage/Brine Mining
149	Observation from Storage/Brine Mining/Oil
150	Observation from Storage/Brine Mining/Gas
151	Observation from Storage/Brine Mining/Oil/Gas
152	Plugged Storage/Brine Mining
153	Plugged Storage/Brine Mining/Oil
154	Plugged Storage/Brine Mining/Gas
155	Plugged Storage/Brine Mining/Oil/Gas 
'''